In [1]:
import sys
import os
src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

In [2]:
from connector import spark_connector

In [3]:
import pandas as pd
import numpy as np
from pyspark.sql import Window
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [4]:
spark_connector?

Init signature:
spark_connector(
    exec_inst: int = 100,
    exec_cores: int = 5,
    aloc_max: int = 500,
    memory_exec: int = 25,
    name: object = 'utopialvo',
    rootdir: object = 'remote_dir_in_hdft',
    enable_graphframes: bool = False,
    enable_clickhouse: bool = False,
    enable_postgres: bool = False,
    dynamic_allocation: bool = False,
    intensive_mode: bool = False,
    marshal_mode: bool = True,
    yarn: bool = True,
) -> None
Docstring:     
Класс стандартного подключения к Spark

exec_inst - количество контейнеров
exec_cores - количество ядер в контейнере
aloc_max - максимальное количество ядер для dynamic_allocation (если он вкл)
memory_exec - количество памяти в контейнере
name - название подключения для упрощения логгирования подключений в кластере
rootdir - корневая папка в HDFS 
enable_graphframes - включить работу с graphframes
enable_clickhouse - включить работу с clickhouse
enable_postgres - включить работу с postgres
dynamic_allocation - включить д

In [5]:
con = spark_connector(exec_inst=100, exec_cores=5, memory_exec=25, enable_graphframes=True, marshal_mode=False, yarn = False)

In [6]:
con.spark

# Metrics

In [7]:
#from metrics import Indices
from metrics2 import Indices

In [8]:
# Тестовые данные
raw_data = con.spark.createDataFrame([
    (1, "A", 1.0, 2.0, 0),
    (2, "B", 1.1, 2.1, 0),
    (3, "C", 5.0, 6.0, 1),
    (4, "D", 5.1, 6.1, 1)
], ["id", "category", "x", "y", "cluster"])


In [9]:
centroids = con.spark.createDataFrame([
    (0, 1.05, 2.05),
    (1, 5.05, 6.05)
], ["cluster_id", "x", 'y'])

In [10]:
from pyspark.storagelevel import StorageLevel

In [11]:
raw_data.repartition(1).persist(StorageLevel.MEMORY_ONLY).foreach(lambda x: x)
centroids.repartition(1).persist(StorageLevel.MEMORY_ONLY).foreach(lambda x: x)

In [15]:
indices = Indices(feature_columns=['x','y'],label_col='cluster',centroid_cluster_col='cluster_id')

In [16]:
%%time
metrics = indices.compute_indices(df= raw_data, centroids_df=centroids, num_clusters=2)

CPU times: user 487 ms, sys: 227 ms, total: 714 ms
Wall time: 2min 54s


In [17]:
metrics

{'Silhouette': 0.9993748047485543,
 'WB-index': 0.0006249999999999984,
 'XU-index': 2.776054458142581,
 'Calinski-Harabasz': 3200.000000000008,
 'WSS': 0.01999999999999995}

In [18]:
from metrics import Indices as Indices2

In [19]:
indices2 = Indices2(feature_columns=['x','y'],label_col='cluster',centroid_cluster_col='cluster_id')

In [20]:
%%time
metrics = indices2.compute_indices(df= raw_data, centroids_df=centroids, num_clusters=2)

CPU times: user 458 ms, sys: 232 ms, total: 690 ms
Wall time: 2min 46s


In [21]:
metrics

{'Silhouette': 0.9993748047485543,
 'WB-index': 0.0006249999999999984,
 'XU-index': 2.776054458142581,
 'Calinski-Harabasz': 3200.000000000008,
 'WSS': 0.01999999999999995}

In [23]:
#con.stop_spark()

time: 2025-02-23 01:43:37.126936
dont del
stop
